In [5]:
import os
import sys
import openai
import json
import requests
import time
from purreaddocs import *


In [7]:
# Load config values

with open(r'C:\\Users\\roshnipatil\\OneDrive - Microsoft\\GitHub_new\\config.json') as config_file:
        config_details = json.load(config_file)
#config_details = readAzAiConfig()
#print(config_details) 

# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
azure_openai_api_key = config_details['OPENAI_API_KEY']
print(azure_openai_api_key)

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

c8d8709f79da4c848586bedae594f86e


In [8]:
openai.api_type = "azure"
openai.api_base = "https://rosh-azai.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = azure_openai_api_key


#readpineconekey
def openai_completion( prompt_input):
    while True:
        try:
            #Azure OpenAI model call 
            response = openai.Completion.create(
                engine="my-davinci-3",
                prompt=prompt_input,
                temperature=1,
                max_tokens=100,
                top_p=0.5,
                frequency_penalty=0,
                presence_penalty=0,
                best_of=1,
                stop=None)
        except openai.error.RateLimitError:
            print(
                "   *** The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.Timeout:
            print(
                "   *** OpenAI API timeout occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIError:
            print(
                    "   *** OpenAI API error occurred. Waiting 10 seconds and trying again. ***"
                )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIConnectionError:
            print(
                "   *** OpenAI API connection error occurred. Check your network settings, proxy configuration, SSL certificates, or firewall rules. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.InvalidRequestError:
            print(
                "   *** OpenAI API invalid request. Check the documentation for the specific API method you are calling and make sure you are sending valid and complete parameters. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.ServiceUnavailableError:
            print(
                "   *** OpenAI API service unavailable. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            print(response.choices[0].text)
            return response.choices[0].text 
            break

output = openai_completion("tell me a joke on animals")
print(output)




Q: What did the fish say when it hit the wall?
A: Dam!


Q: What did the fish say when it hit the wall?
A: Dam!


In [ ]:
# Request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/completions?api-version={openai_api_version}"

# Example prompt for request payload
prompt = "Hello world"

# Json payload
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
json_data = {
  "prompt": prompt,
  "temperature":0,
  "max_tokens": 30
}

# Including the api-key in HTTP headers
print (openai_api_key)
headers =  {"api-key": openai_api_key}
print (api_url)
try:
    # Request for creating a completion for the provided prompt and parameters
    response = requests.post(api_url, json=json_data, headers=headers)
    completion = response.json()
    
    # print the completion
    print(completion['choices'][0]['text'])
    
    # Here indicating if the response is filtered
    if completion['choices'][0]['finish_reason'] == "content_filter":
        print("The generated content is filtered.")
except:
    print("An exception has occurred. \n")
    print("Error Message:", completion['error']['message'])

c8d8709f79da4c848586bedae594f86e
https://rosh-azai.openai.azure.com/openai/deployments/my-davinci-3/completions?api-version=2022-12-01


Hello! Welcome to the world!


In [ ]:

llm = AzureOpenAI(openai_api_key=openai_api_key, deployment_name=deployment_name, openai_api_version= openai_api_version, model_name="text-davinci-003", openai_api_base=openai_api_base )


contents = readSampleDoc()

myprompt1 = f"""
identify regex pattern ^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$ in the text and return name value in JSON format with name as mycustclass ?

{contents}
"""

llm_out1 = llm( myprompt1 )
print (llm_out1)

{ "mycustclass": "roshnip@microsoft.com" }


In [ ]:
#SQL Query
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

with open(r'C:\\Users\\roshnipatil\\Documents\\GitHub\\configdb.json') as config_file:
        config_sql_details = json.load(config_file)

database_user = config_sql_details["DATABASE_USERNAME"]
mydatabase_password = config_sql_details["DATABASE_PASSWORD"]
print (mydatabase_password)
database_server = config_sql_details["DATABASE_SERVER"]
database_db = config_sql_details["DATABASE_DB"]


ayK80aR0YZ


In [ ]:
import pyodbc, struct
import pandas as pd
from azure import identity

from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

#Silent connect using AD user
db_str1 = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:rosh1dbsrv1.database.windows.net,1433;Database=rosh1db11;Uid='+ database_user+ ';Pwd='+mydatabase_password+';Authentication=ActiveDirectoryPassword; Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
#print (db_str1)
cnxn = pyodbc.connect(db_str1)


#Connect using provided AD user
#connection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:rosh1dbsrv1.database.windows.net,1433;Database=rosh1db11;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'
#credential = identity.DefaultAzureCredential(exclude_interactive_browser_credential=False)
#token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
#token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
#SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h
#cnxn = pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})



#cnxn = pyodbc.connect(DRIVER={ODBC Driver 17 for SQL Server}; SERVER=database_server; DATABASE=database_db;ENCRYPT=yes; UID=database_user; PWD=mydatabase_password)
#cnxn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};S=tcp:database_server,1433;Uid=database_user;Pwd=database_password;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;")
cursor = cnxn.cursor()
# select 10 rows from SQL table to insert in dataframe.
query = "select * from [SalesLT].[Address];"
#df = pd.read_sql(query, cnxn)
#print(df.head(10))

#from langchain.agents import create_pandas_dataframe_agent
#agent = create_pandas_dataframe_agent(llm, df, verbose=True)
#agent.run("how many rows are there?")

cnxn.close()
del cnxn

In [ ]:
from langchain.sql_database import SQLDatabase

connection_string = f"mssql+pymssql://{database_user}:{mydatabase_password}@{database_server}:1433/{database_db}/"
print (connection_string)
db = SQLDatabase.from_uri(connection_string)